# Forge Authentication Sample Notebook

In [ ]:
# Python standard library dependencies
import json
import urllib.parse as urlparse
from urllib.parse import parse_qs

# External dependencies (pip install)
import requests

## Forge Application Details

In [ ]:
# Forge app credentials (optionally load from a local json config file)
client_id = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client_secret = "XXXXXXXXXXXXXXXX"
callback_url = "http://localhost:8888/" # Update on your app for testing in a Jupyter notebook

## 2-Legged Token Sample

In [ ]:
# Request an access token
url = "https://developer.api.autodesk.com/authentication/v1/authenticate"

data = {"client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"}

headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "application/json*"}

request = requests.post(url, data=data, headers=headers)

print(request.status_code, request.reason)

In [ ]:
# Handle various status code states for Forge
code = request.status_code

if(code == 200):
    print("Successful request; access token returned." + "\n")
    
    token_object = json.loads(request.text)
    
    for key in token_object:
        print(key + ": " + str(token_object[key]) + "\n")
    
elif(code == 400):
    print("One or more parameters are invalid. Examine the response payload body for details.")
elif(code == 401):
    print("The client_id and client_secret combination is not valid.")
elif(code == 403):
    print("The client_id is not authorized to access this endpoint.")
elif(code == 415):
    print("The Content-Type header is missing or specifies a value other than application/x-www-form-urlencoded.")
elif(code == 429):
    print("Rate limit exceeded; wait some time before retrying.")
elif(code == 500):
    print("Generic internal server error.")
else:
    print("Unhandle " + str(code) + " error.")

## 3-Legged Token with Authorization Code Grant
### (Sign in with Autodesk ID)

In [ ]:
# Request user to login
url = "https://developer.api.autodesk.com/authentication/v1/authorize"

data = {"client_id": client_id, 
        "response_type": "code", 
        "redirect_uri": callback_url, 
        "scope": "data:read"}

req = requests.Request("GET", url, params=data)
prep = req.prepare()
r = requests.get(prep.url)
print(prep.url)

print(r.status_code)

#### Click the link above and copy the full URL from the new window/tab to the 'response_url' variable below (after signing in - if not already logged in to Forge).  This will simulate the callback which contains the appended authorization code required to obtain a 3-legged token.

In [ ]:
# Display a dialog box reminding the user to update the response_url
from IPython.display import display, Javascript
display(Javascript("""
require(
    ["base/js/dialog"], 
    function(dialog) {
        dialog.modal({
            title: 'Reminder',
            body: 'Did you make sure to copy/paste the url from the new browser tab launched by click the link in the previous cell into the cell below?',
            buttons: {
                'Yes': {}
            }
        });
    }
);
"""))

In [ ]:
# After logging in the user will recieve a url in the following format with a "code" appended
# Paste the full url below, such as:
response_url = "http://localhost:8888/tree?code=4_nN1nd-v8HBgjho_NrcZkQ4TrekTSG50Ps5m-P3"

In [ ]:
# Retrieve the code from the url
parsed = urlparse.urlparse(response_url)
auth_code = parse_qs(parsed.query)["code"][0]

print(auth_code)

In [ ]:
# Request an access token
url = "https://developer.api.autodesk.com/authentication/v1/gettoken"

data = {"client_id": client_id,
        "client_secret": client_secret, 
        "grant_type": "authorization_code", 
        "code": auth_code, 
        "redirect_uri": callback_url}

headers = {"Content-type": "application/x-www-form-urlencoded", 
           "Accept": "application/json*"}

request = requests.post(url, data=data, headers=headers)

print(request.status_code, request.reason)

In [ ]:
code = request.status_code

if(code == 200):
    print("Successful request; access token returned." + "\n")
    
    token_object = json.loads(request.text)
    
    for key in token_object:
        print(key + ": " + str(token_object[key]) + "\n") 

elif(code == 400):
    print("One or more parameters are invalid. Examine the response payload body for details.")
elif(code == 401):
    print("The client_id and client_secret combination is not valid.")
elif(code == 403):
    print("The client_id is not authorized to access this endpoint.")
elif(code == 415):
    print("The Content-Type header is missing or specifies a value other than application/x-www-form-urlencoded.")
elif(code == 429):
    print("Rate limit exceeded; wait some time before retrying.")
elif(code == 500):
    print("Generic internal server error.")
else:
    print("Unhandle " + str(code) + " error.")

## Get User Credentials

In [ ]:
url = "https://developer.api.autodesk.com/userprofile/v1/users/@me"
headers = {"Authorization": token_object["token_type"]+ " " + token_object["access_token"]}

request = requests.get(url, headers=headers)

print(request.status_code, request.reason)

In [ ]:
if(request.status_code == 200):
    print("Successful request; user data returned." + "\n")
    
    user_object = json.loads(request.text)
    
    for key in user_object:
        print(key + ": " + str(user_object[key]) + "\n")

elif(code == 401):
    print("The access token is invalid.")
elif(code == 429):
    print("Rate limit exceeded; wait some time before retrying.")
elif(code == 500):
    print("Generic internal server error.")
else:
    print("Unhandle " + str(code) + " error.")